# CHIRPS V3 Data Extractor

Extract CHIRPS V3.0 precipitation data for specific coordinates and date ranges.

In [2]:
# Configuration Constants
LATITUDE = 42.0
LONGITUDE = -93.5
START_DATE = '2020-01-01'
END_DATE = '2020-01-31'
DATA_DIR = './chirps_v3_data'
CHIRPS_V3_BASE_URL = 'https://data.chc.ucsb.edu/products/CHIRPS/v3.0/daily/final/rnl'

In [ ]:
import os
import requests
import pandas as pd
import rioxarray
from datetime import datetime
from calendar import monthrange
from pathlib import Path

In [ ]:
def check_existing_files(start_date, end_date, data_dir):
    """Check which CHIRPS V3 files already exist locally."""
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    
    Path(data_dir).mkdir(parents=True, exist_ok=True)
    
    existing_files = []
    missing_dates = []
    
    current = start
    while current <= end:
        year = current.year
        month = current.month
        day = current.day
        
        filename = f'chirps-v3.0.rnl.{year}.{month:02d}.{day:02d}.tif'
        filepath = Path(data_dir) / filename
        
        if filepath.exists():
            existing_files.append(filepath)
        else:
            missing_dates.append(current)
        
        current += pd.Timedelta(days=1)
    
    total_days = (end - start).days + 1
    return {
        'existing': existing_files,
        'missing': missing_dates,
        'total': total_days,
        'existing_count': len(existing_files),
        'missing_count': len(missing_dates)
    }

In [ ]:
def download_chirps_v3(missing_dates, data_dir, base_url):
    """Download missing CHIRPS V3 TIF files."""
    downloaded = []
    failed = []
    
    for date in missing_dates:
        year = date.year
        month = date.month
        day = date.day
        
        filename = f'chirps-v3.0.rnl.{year}.{month:02d}.{day:02d}.tif'
        filepath = Path(data_dir) / filename
        url = f'{base_url}/{year}/{filename}'
        
        try:
            response = requests.get(url, stream=True, timeout=300)
            response.raise_for_status()
            
            with open(filepath, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
            
            downloaded.append(filepath)
            
        except Exception as e:
            failed.append((filename, str(e)))
    
    return {'downloaded': downloaded, 'failed': failed}

In [ ]:
def load_chirps_data(file_paths, lat, lon):
    """Load CHIRPS V3 data and extract values for specific coordinates."""
    data = []
    
    for filepath in sorted(file_paths):
        filename = filepath.name
        parts = filename.split('.')
        year = int(parts[3])
        month = int(parts[4])
        day = int(parts[5])
        
        da = rioxarray.open_rasterio(filepath, masked=True)
        da = da.squeeze().drop_vars('band', errors='ignore')
        
        point_value = da.sel(x=lon, y=lat, method='nearest').values
        
        if hasattr(point_value, 'item'):
            point_value = point_value.item()
        
        if point_value < 0 or pd.isna(point_value):
            point_value = 0.0
        
        data.append({
            'year': year,
            'month': month,
            'day': day,
            'precip': float(point_value)
        })
        
        da.close()
    
    return data

In [ ]:
def create_dataframe(data):
    """Convert data to DataFrame with DATE (yyyyddd) and CRAIN columns."""
    df = pd.DataFrame(data)
    
    df['date'] = pd.to_datetime(df[['year', 'month', 'day']])
    df['day_of_year'] = df['date'].dt.dayofyear
    df['DATE'] = df['year'] * 1000 + df['day_of_year']
    df['CRAIN'] = df['precip']
    
    result = df[['DATE', 'CRAIN']].copy()
    
    return result

## Execution

In [ ]:
file_status = check_existing_files(START_DATE, END_DATE, DATA_DIR)
file_status

In [ ]:
if file_status['missing_count'] > 0:
    download_result = download_chirps_v3(file_status['missing'], DATA_DIR, CHIRPS_V3_BASE_URL)
    download_result
else:
    {'message': 'All files already exist'}

In [ ]:
all_files = check_existing_files(START_DATE, END_DATE, DATA_DIR)['existing']
raw_data = load_chirps_data(all_files, LATITUDE, LONGITUDE)
raw_data[:5]

In [ ]:
chirps_df = create_dataframe(raw_data)
chirps_df